In [7]:
import os
import shutil
import joblib
import librosa
import numpy as np
import pandas as pd

from pyAudioAnalysis import audioBasicIO, ShortTermFeatures

In [2]:

BASE_DIR            = os.getcwd()

FOLDS               = ['Fold 1', 'Fold 2', 'Fold 3', 'Fold 4', 'Fold 5']
EMOTIONS            = ['Approval', 'Disapproval', 'Neutral']

AUDIO_DIR_AF      = '../../Dataset/Audio-AF/'
AUDIO_DIR_VAL     = '../../Dataset/Audio-VAL/'

FEATURES_DIR_AF     = '../../Dataset/Features-AF/'
FEATURES_DIR_VAL    = '../../Dataset/Features-VAL/'


In [3]:
write_dir = os.path.join(BASE_DIR, FEATURES_DIR_AF)

print('cleaning already existing files ... ', end='')
try:
    shutil.rmtree(write_dir)
    print('√')
except:
    print('✕')
    
print('creating windows directory ... ', end='')
os.mkdir(write_dir)
print('√')


path = os.path.join(BASE_DIR, AUDIO_DIR_AF)

X = pd.DataFrame()

for emotion in EMOTIONS:
    print('Processing data for ' + emotion, end=' ... ')
    
    for fold in FOLDS:
        fold_path = os.path.join(path, emotion, fold)
        filenames = os.listdir(fold_path)
        
        for filename in filenames:
            file_path = os.path.join(fold_path, filename)
            [Fs, x] = audioBasicIO.read_audio_file(file_path)
            F, f_names = ShortTermFeatures.feature_extraction(
                x, Fs, 1.0*Fs, 0.50*Fs, deltas=False
            )
            
            clip_features = pd.DataFrame(F.T, columns=f_names)
            
            e_idx = EMOTIONS.index(emotion)
            y = pd.DataFrame(np.ones((clip_features.shape[0], 1)) * e_idx, columns=['label'])
            
            f_idx = FOLDS.index(fold)
            f = pd.DataFrame(np.ones((clip_features.shape[0], 1)) * f_idx, columns=['fold'])
            
            clip_features = pd.concat([clip_features, y, f], axis=1)
            
            X = pd.concat([X, clip_features], ignore_index=True)
        
    print('√')
    
features_path = os.path.join(write_dir, 'Features.joblib')
joblib.dump(X, features_path)

cleaning already existing files ... √
creating windows directory ... √
Processing data for Approval ... √
Processing data for Disapproval ... √
Processing data for Neutral ... √


['/home/andromeda/Python/Crowd-Emotion/src/classical-ml/../../Dataset/Features-AF/Features.joblib']

In [8]:
write_dir = os.path.join(BASE_DIR, FEATURES_DIR_VAL)

print('cleaning already existing files ... ', end='')
try:
    shutil.rmtree(write_dir)
    print('√')
except:
    print('✕')
    
print('creating windows directory ... ', end='')
os.mkdir(write_dir)
print('√')


path = os.path.join(BASE_DIR, AUDIO_DIR_VAL)

X = pd.DataFrame()

for emotion in EMOTIONS:
    print('Processing data for ' + emotion, end=' ... ')
    
    emotion_path = os.path.join(path, emotion)
    filenames = os.listdir(emotion_path)

    for filename in filenames:
        file_path = os.path.join(emotion_path, filename)
        x, Fs = librosa.load(file_path, sr=44100)
        F, f_names = ShortTermFeatures.feature_extraction(
            x, Fs, 1.0*Fs, 0.50*Fs, deltas=False
        )

        clip_features = pd.DataFrame(F.T, columns=f_names)

        e_idx = EMOTIONS.index(emotion)
        y = pd.DataFrame(np.ones((clip_features.shape[0], 1)) * e_idx, columns=['label'])

        f_idx = FOLDS.index(fold)
        f = pd.DataFrame(np.ones((clip_features.shape[0], 1)) * f_idx, columns=['fold'])

        clip_features = pd.concat([clip_features, y, f], axis=1)

        X = pd.concat([X, clip_features], ignore_index=True)
        
    print('√')
    
features_path = os.path.join(write_dir, 'Features.joblib')
joblib.dump(X, features_path)

cleaning already existing files ... √
creating windows directory ... √
Processing data for Approval ... √
Processing data for Disapproval ... √
Processing data for Neutral ... √


['/home/andromeda/Python/Crowd-Emotion/src/classical-ml/../../Dataset/Features-VAL/Features.joblib']